# Polar Decomposition (Linear Algebra) — Theory + Computation


This notebook explains the **polar decomposition** of a matrix and shows how to compute it in practice with NumPy.

## Learning goals
- State what polar decomposition is and what it means geometrically.
- Distinguish **left** and **right** polar decompositions.
- Compute polar factors using the **SVD**.
- Verify the decomposition numerically and understand uniqueness conditions.



## 1. Definition

Let $A \in \mathbb{R}^{m \times n}$ (everything here also works over $\mathbb{C}$ with conjugate-transpose).

### Right polar decomposition
There exists a factorization
$$
A = Q\,H
$$
where

- $Q \in \mathbb{R}^{m \times n}$ has **orthonormal columns**, i.e. $Q^TQ = I_n$ (when $m \ge n$).  
  So $Q$ is a **partial isometry** (orthogonal if $m=n$).
- $H \in \mathbb{R}^{n \times n}$ is **symmetric positive semidefinite (PSD)**:  
  $H = H^T$ and $x^T H x \ge 0$ for all $x$.

### Left polar decomposition
There is also a factorization
$$
A = K\,Q
$$
where
- $K \in \mathbb{R}^{m \times m}$ is symmetric PSD,
- $Q$ is as above.

For square matrices ($m=n$), both exist and are closely related.



## 2. Geometric meaning (square case)

For $A \in \mathbb{R}^{n \times n}$ invertible:
$$
A = QH
$$
can be read as:

- $H$ applies a **pure stretch** (no rotation/reflection), because it is symmetric PSD.
- $Q$ applies a **rigid motion** (rotation/reflection), because it is orthogonal.

Analogy with real numbers $a\ne 0$:
$$
a = \operatorname{sign}(a)\,|a|.
$$



## 3. Existence and uniqueness (key facts)

### Existence
Every matrix $A$ admits a polar decomposition.

### Canonical PSD factor
For the right polar decomposition, the canonical choice is
$$
H = (A^T A)^{1/2},
$$
the **principal** (PSD) square root of the PSD matrix $A^TA$.

Then one can define
$$
Q = A\,H^{\dagger},
$$
where $H^{\dagger}$ is the Moore–Penrose pseudoinverse (needed when $A$ is rank-deficient).

### Uniqueness
- $H=(A^TA)^{1/2}$ is **unique** (principal square root).
- $Q$ is **unique if $A$ has full column rank**; otherwise $Q$ is not unique (freedom on the null space).



## 4. Computing polar decomposition via SVD

Let the singular value decomposition be
$$
A = U \Sigma V^T,
$$
with singular values $\sigma_i \ge 0$ on $\Sigma$.

### Right polar decomposition
A standard choice is
$$
Q = U V^T,\qquad H = V \Sigma V^T.
$$

### Left polar decomposition
Similarly,
$$
K = U \Sigma U^T,\qquad Q = U V^T.
$$

SVD is typically the most numerically robust approach.


In [ ]:
import numpy as np

np.set_printoptions(precision=4, suppress=True)

def polar_decomposition_right(A: np.ndarray):
    """
    Right polar decomposition A = Q H
    - Q has orthonormal columns (partial isometry)
    - H is symmetric positive semidefinite (n x n)
    Works for any m x n matrix.
    """
    U, s, Vt = np.linalg.svd(A, full_matrices=False)  # U: m x r, Vt: r x n
    Q = U @ Vt                                       # m x n
    V = Vt.T                                         # n x r
    Sigma = np.diag(s)                               # r x r
    H = V @ Sigma @ V.T                              # n x n
    return Q, H

def polar_decomposition_left(A: np.ndarray):
    """
    Left polar decomposition A = K Q
    - K is symmetric positive semidefinite (m x m)
    - Q has orthonormal columns (partial isometry)
    """
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    Q = U @ Vt
    Sigma = np.diag(s)
    K = U @ Sigma @ U.T
    return K, Q

def is_symmetric(M, tol=1e-10):
    return np.linalg.norm(M - M.T, ord='fro') <= tol

def is_psd(M, tol=1e-10):
    w = np.linalg.eigvalsh((M + M.T) / 2)
    return (np.min(w) >= -tol), w



## 5. Example 1 — Square, invertible matrix

In the square full-rank case ($A\in\mathbb{R}^{n\times n}$ invertible):
- $Q$ will be orthogonal,
- $H$ will be symmetric **positive definite** (PD),
- the factorization is unique.


In [ ]:
A = np.array([[2.0, 1.0],
              [1.0, 3.0]])

Q, H = polar_decomposition_right(A)

print("A:\n", A)
print("\nQ:\n", Q)
print("\nH:\n", H)

print("\nCheck A ≈ QH:", np.allclose(A, Q @ H))
print("Q^T Q ≈ I:", np.allclose(Q.T @ Q, np.eye(Q.shape[1])))
print("H symmetric:", is_symmetric(H))
psd_ok, eigs = is_psd(H)
print("H PSD:", psd_ok, "eigs:", eigs)
print("det(Q) =", np.linalg.det(Q))



## 6. Example 2 — Rectangular matrix ($m>n$)

Here $A \in \mathbb{R}^{m\times n}$ with $m>n$.

In $A = QH$:
- $Q$ has orthonormal columns ($Q^TQ=I_n$),
- $H$ is $n\times n$ PSD.


In [ ]:
A = np.array([[1.0, 0.0],
              [2.0, 1.0],
              [0.0, 2.0]])

Q, H = polar_decomposition_right(A)

print("A shape:", A.shape)
print("Q shape:", Q.shape)
print("H shape:", H.shape)

print("\nCheck A ≈ QH:", np.allclose(A, Q @ H))
print("Q^T Q ≈ I:", np.allclose(Q.T @ Q, np.eye(Q.shape[1])))
print("H symmetric:", is_symmetric(H))
psd_ok, eigs = is_psd(H)
print("H PSD:", psd_ok, "eigs:", eigs)



## 7. Example 3 — Rank-deficient matrix (non-uniqueness of $Q$)

If $A$ is rank-deficient, $H$ is still uniquely defined as $(A^TA)^{1/2}$ (principal PSD root),
but $Q$ is generally **not unique**.

The SVD-based construction produces a standard, well-defined choice.


In [ ]:
A = np.array([[1.0, 2.0],
              [2.0, 4.0]])  # rank 1

Q, H = polar_decomposition_right(A)

print("A:\n", A)
print("\nSingular values:", np.linalg.svd(A, compute_uv=False))
print("\nCheck A ≈ QH:", np.allclose(A, Q @ H))
print("Q^T Q ≈ I:", np.allclose(Q.T @ Q, np.eye(Q.shape[1])))
psd_ok, eigs = is_psd(H)
print("H PSD:", psd_ok, "eigs:", eigs)



## 8. Relation to $A^TA$ and the matrix square root

A central identity (right polar decomposition) is:
$$
H^2 = A^T A.
$$
So $H@H$ should match $A.T@A$ up to floating-point error.


In [ ]:
A = np.array([[3.0, 1.0],
              [0.0, 2.0],
              [1.0, 1.0]])

Q, H = polar_decomposition_right(A)

AtA = A.T @ A
H2 = H @ H

print("||H^2 - A^T A||_F =", np.linalg.norm(H2 - AtA, ord='fro'))
print("Allclose:", np.allclose(H2, AtA))



## 9. Left vs right polar decomposition (square case)

For square $A$:
- Right: $A = QH$ with $H = (A^TA)^{1/2}$.
- Left:  $A = KQ$ with $K = (AA^T)^{1/2}$.

They share the same orthogonal factor $Q$ (for invertible $A$), while the PSD factor moves sides.


In [ ]:
A = np.array([[1.0, 2.0, 0.0],
              [0.0, 1.0, 1.0],
              [2.0, 0.0, 1.0]])

Q, H = polar_decomposition_right(A)
K, Q2 = polar_decomposition_left(A)

print("Right:  ||A - QH||_F =", np.linalg.norm(A - Q @ H, ord='fro'))
print("Left:   ||A - KQ||_F =", np.linalg.norm(A - K @ Q2, ord='fro'))
print("Q factors close:", np.allclose(Q, Q2))

print("\nH symmetric:", is_symmetric(H), "   K symmetric:", is_symmetric(K))
psdH, eigH = is_psd(H); psdK, eigK = is_psd(K)
print("H PSD:", psdH, "eigs(min..max):", (eigH.min(), eigH.max()))
print("K PSD:", psdK, "eigs(min..max):", (eigK.min(), eigK.max()))



## 10. Where polar decomposition is used

- **Nearest orthogonal matrix**: $Q$ is the closest orthogonal matrix to $A$ in Frobenius norm.
- **Computer graphics / vision**: separating rotation from deformation.
- **Continuum mechanics**: rotation tensor and stretch tensor.
- **Orthogonalization**: stable way to extract an orthonormal factor.

---

## 11. Exercises (optional)

1. Generate random square matrices and inspect $\det(Q)$.
2. Verify numerically that $Q$ solves:
   $$
   \min_{R^TR=I} \|A - R\|_F.
   $$


In [ ]:
# Exercise 1 starter: random trials for det(Q) in square case
rng = np.random.default_rng(0)

for k in range(5):
    A = rng.normal(size=(3,3))
    Q, H = polar_decomposition_right(A)
    print(k, "det(Q) =", np.linalg.det(Q), "  ||A-QH||_F =", np.linalg.norm(A - Q@H))
